In [1]:
import csv
from pprint import pprint
#importing punctuation models in python
from string import punctuation as p
#import regex
import re
import sys
import os

#arg1 = 'tweet_kotor.csv'
#arg2 = 'tweet_bersih.csv'

arg1 = sys.argv[1]
arg2 = sys.argv[2]


def processtweet(tweet):
    #process the tweets
    
    #convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    #delete username*
    tweet = re.sub('username[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    #punctuation
    tweet = re.sub('[0123456789\!#$%&()*+,-./:;<=>?@^_`{|}~]','',tweet)
    #special case for ' " [ ]
    tweet = re.sub("'","",tweet)
    tweet = re.sub('"','',tweet)
    tweet = re.sub("\[","",tweet)
    tweet = re.sub("\]","",tweet)
    #delete all unicode
    tweet = re.sub(r'[^\x00-\x7F]+','',tweet)
    return tweet
 
maxInt = sys.maxsize
csv.field_size_limit(maxInt)
with open(arg1,'rU') as csv_file:
    with open(arg2,'wb') as csv_result:
        reader = csv.reader(csv_file, delimiter=',')
	#next(reader,None) #skipheader
        writer = csv.writer(csv_result)

        
        for line in reader:
	    tweet_array = []

            tweets = line[1]
            processedTweet = processtweet(tweets)
            tweet_array.append(processedTweet)
	    #print processedTweet
	    #print tweets
            print tweet_array
	    writer.writerow(tweet_array)

IOError: [Errno 2] No such file or directory: '-f'

In [2]:
import json
from pprint import pprint
import os
import re
import glob
import pandas as pd

#==============================================================================
# with open('data.json') as data_file:    
#     data = json.load(data_file)
#==============================================================================
    

def extract_one_file(json_path):
    json_clean = {}
    instagram_path = 'www.instagram.com/p'
    with open(json_path) as data_file:    
        data = json.load(data_file)
    
    url = os.path.join(instagram_path,data['shortcode'])
    like_count = data['edge_media_preview_like']['count']
    comment_count = data['edge_media_to_comment']['count']
    comment_text= []
    comment_user = []
    for item in data['edge_media_to_comment']['edges']:
        comment_user.append(item['node']['owner']['username'])
        comment_text.append(item['node']['text'])
    #comment_user_list = [{'user' : item['node']['owner']['username'],'comment':item['node']['text']} for item in data['edge_media_to_comment']['edges']]
    media = data['display_url']
    caption = data['edge_media_to_caption']['edges'][0]['node']['text']
    hashtag = re.findall(r"#(\w+)", caption)
    instagram_id = data['owner']['username']
    instagram_alias = data['owner']['full_name']
    json_clean['url']=url
    json_clean['like_count'] = like_count
    json_clean['comment_count'] = comment_count
    json_clean['comment_acount'] = comment_user
    json_clean['comment_text'] = comment_text
    json_clean['media'] = media
    json_clean['caption'] = caption
    json_clean['hashtag'] = hashtag
    json_clean['instagram_id'] = instagram_id
    json_clean['instagram_alias'] = instagram_alias
    return json_clean


def extract_from_folder(json_folder_path):
    json_list = glob.glob(json_folder_path+'/*')
    json_list = []
    for json_file in json_list:
        json_list.append(extract_one_file(json_file))
        
    return json_list

if __name__ == '__main__':
    json_file = 'data.json'
    filename = 'tes.xlsx'
    #json_folder = 'pki_2017_09'
    json1 = extract_one_file(json_file)
    #json_folder = extract_from_folder(json_folder)
    df = pd.DataFrame({k : pd.Series(v) for k, v in json1.iteritems()})
    df.to_excel(filename)

IOError: [Errno 2] No such file or directory: 'data.json'